# Task 2: Training and Tuning with Ray

## Part 1: Training with Ray Train and Xgboost
In this task, you will train a machine learning model using the preprocessed data. The goal is to train an Xgboost model to predict the user rating for a product. 

In [1]:
# !pip install ray==2.50.0 --quiet
# !pip install xgboost==1.7.6 --quiet
# !pip install -U tensorboardx --quiet

In [2]:
import ray
import psutil
ray.shutdown()
NINE_GIB = 9 * 1024 * 1024 * 1024
ray.init(object_store_memory=int(NINE_GIB))
import pandas as pd
import os
import json
import random
import numpy as np
from pathlib import Path
seed = 41
random.seed(seed)
np.random.seed(seed)

from ray.train.xgboost import XGBoostTrainer
from ray.train import ScalingConfig, RunConfig

2025-11-20 22:33:44,977	INFO worker.py:2004 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
/home/wew068/.local/lib/python3.11/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
(XGBoostTrainer pid=42707) Started distributed worker processes: 
(XGBoostTrainer pid=42707) - (node_id=68b8eea9288f716a1d135c3f1856321e6a4d3ecbe0672f1ef55cd21a, ip=10.32.73.154, pid=42910) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=42910) [22:33:54] task [xgboost.ray-rank=00000000]:eb8829f4171ce3dc05d976a501000000 got new rank 0
(SplitCoordinator pid=43041) Registered dataset logger for dataset train_1_0
(SplitCoordinator pid=43041) Starting execution of Dataset train_1_0. Full logs are in /tmp/ray/session_20

(XGBoostTrainer pid=42707) [0]	train-rmse:2.79975
(XGBoostTrainer pid=42707) [1]	train-rmse:2.11608
(XGBoostTrainer pid=42707) [2]	train-rmse:1.68245
(XGBoostTrainer pid=42707) [3]	train-rmse:1.42241
(XGBoostTrainer pid=42707) [4]	train-rmse:1.27567
(XGBoostTrainer pid=42707) [5]	train-rmse:1.19713
(XGBoostTrainer pid=42707) [6]	train-rmse:1.15659
(XGBoostTrainer pid=42707) [7]	train-rmse:1.13610
(XGBoostTrainer pid=42707) [8]	train-rmse:1.12581
(XGBoostTrainer pid=42707) [9]	train-rmse:1.12071


(RayTrainWorker pid=42910) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-33-48/XGBoostTrainer_fb7d8_00000_0_2025-11-20_22-33-50/checkpoint_000000)
(RayTrainWorker pid=42910) Registered dataset logger for dataset dataset_3_0
(XGBoostTrainer pid=44755) Started distributed worker processes: 
(XGBoostTrainer pid=44755) - (node_id=68b8eea9288f716a1d135c3f1856321e6a4d3ecbe0672f1ef55cd21a, ip=10.32.73.154, pid=44887) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=44887) [22:57:00] task [xgboost.ray-rank=00000000]:a1ca4b2421989f82908eab7501000000 got new rank 0
(SplitCoordinator pid=45016) Registered dataset logger for dataset train_9_0
(SplitCoordinator pid=45016) Starting execution of Dataset train_9_0. Full logs are in /tmp/ray/session_2025-11-20_22-33-41_618501_40643/logs/ray-data
(SplitCoordinator pid=45016) Execution plan of Dataset train_9_0: InputDataBuffer[Input] -> OutputSplitter[split(

(XGBoostTrainer pid=44755) [0]	train-rmse:2.79972	validation-rmse:2.79912
(XGBoostTrainer pid=44755) [1]	train-rmse:2.11589	validation-rmse:2.11562
(XGBoostTrainer pid=44755) [2]	train-rmse:1.68215	validation-rmse:1.68226
(XGBoostTrainer pid=44755) [3]	train-rmse:1.42199	validation-rmse:1.42251
(XGBoostTrainer pid=44755) [4]	train-rmse:1.27514	validation-rmse:1.27600
(XGBoostTrainer pid=44755) [5]	train-rmse:1.19658	validation-rmse:1.19771
(XGBoostTrainer pid=44755) [6]	train-rmse:1.15603	validation-rmse:1.15737
(XGBoostTrainer pid=44755) [7]	train-rmse:1.13557	validation-rmse:1.13707
(XGBoostTrainer pid=44755) [8]	train-rmse:1.12528	validation-rmse:1.12684
(XGBoostTrainer pid=44755) [9]	train-rmse:1.12009	validation-rmse:1.12173


(RayTrainWorker pid=44887) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-56-55/XGBoostTrainer_35e72_00000_0_eta=0.3000,max_depth=3_2025-11-20_22-56-55/checkpoint_000000)
(RayTrainWorker pid=44887) Registered dataset logger for dataset dataset_14_0 [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(SplitCoordinator pid=45017) Starting execution of Dataset validation_11_0. Full logs are in /tmp/ray/session_2025-11-20_22-33-41_618501_40643/logs/ray-data
(SplitCoordinator pid=45017) Execution plan of Dataset validation_11_0: InputDataBuffer[Input] -> OutputSplitter[split(1, equal=True)]
(SplitCoordinator pid=45017) ✔️  Dataset validation_11_0 execution finished in 0.18 seconds
(XGBoostTrainer pid=45311) Started distr

...

(RayTrainWorker pid=45443) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-56-55/XGBoostTrainer_35e72_00001_1_eta=0.5000,max_depth=3_2025-11-20_22-57-13/checkpoint_000000)
(RayTrainWorker pid=45443) Registered dataset logger for dataset dataset_20_0 [repeated 3x across cluster]
(SplitCoordinator pid=45574) Starting execution of Dataset validation_17_0. Full logs are in /tmp/ray/session_2025-11-20_22-33-41_618501_40643/logs/ray-data
(SplitCoordinator pid=45574) Execution plan of Dataset validation_17_0: InputDataBuffer[Input] -> OutputSplitter[split(1, equal=True)]
(SplitCoordinator pid=45574) ✔️  Dataset validation_17_0 execution finished in 0.16 seconds
(XGBoostTrainer pid=45865) Started distributed worker processes: 
(XGBoostTrainer pid=45865) - (node_id=68b8eea9288f716a1d135c3f1856321e6a4d3ecbe0672f1ef55cd21a, ip=10.32.73.154, pid=45995) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=459

(XGBoostTrainer pid=45865) [0]	train-rmse:2.79953	validation-rmse:2.79897
(XGBoostTrainer pid=45865) [1]	train-rmse:2.11524	validation-rmse:2.11504
(XGBoostTrainer pid=45865) [2]	train-rmse:1.68094	validation-rmse:1.68119
(XGBoostTrainer pid=45865) [3]	train-rmse:1.42031	validation-rmse:1.42099
(XGBoostTrainer pid=45865) [4]	train-rmse:1.27271	validation-rmse:1.27380
(XGBoostTrainer pid=45865) [5]	train-rmse:1.19356	validation-rmse:1.19496
(XGBoostTrainer pid=45865) [6]	train-rmse:1.15248	validation-rmse:1.15412
(XGBoostTrainer pid=45865) [7]	train-rmse:1.13184	validation-rmse:1.13365
(XGBoostTrainer pid=45865) [8]	train-rmse:1.12122	validation-rmse:1.12316
(XGBoostTrainer pid=45865) [9]	train-rmse:1.11575	validation-rmse:1.11781


(RayTrainWorker pid=45995) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-56-55/XGBoostTrainer_35e72_00002_2_eta=0.3000,max_depth=5_2025-11-20_22-57-32/checkpoint_000000)
(RayTrainWorker pid=45995) Registered dataset logger for dataset dataset_26_0 [repeated 3x across cluster]
(SplitCoordinator pid=46127) Starting execution of Dataset validation_23_0. Full logs are in /tmp/ray/session_2025-11-20_22-33-41_618501_40643/logs/ray-data
(SplitCoordinator pid=46127) Execution plan of Dataset validation_23_0: InputDataBuffer[Input] -> OutputSplitter[split(1, equal=True)]
(SplitCoordinator pid=46127) ✔️  Dataset validation_23_0 execution finished in 0.13 seconds
(XGBoostTrainer pid=46417) Started distributed worker processes: 
(XGBoostTrainer pid=46417) - (node_id=68b8eea9288f716a1d135c3f1856321e6a4d3ecbe0672f1ef55cd21a, ip=10.32.73.154, pid=46548) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=465

(XGBoostTrainer pid=46417) [0]	train-rmse:2.14681	validation-rmse:2.14654
(XGBoostTrainer pid=46417) [1]	train-rmse:1.44358	validation-rmse:1.44410
(XGBoostTrainer pid=46417) [2]	train-rmse:1.20447	validation-rmse:1.20574
(XGBoostTrainer pid=46417) [3]	train-rmse:1.13627	validation-rmse:1.13804
(XGBoostTrainer pid=46417) [4]	train-rmse:1.11819	validation-rmse:1.12012
(XGBoostTrainer pid=46417) [5]	train-rmse:1.11277	validation-rmse:1.11486
(XGBoostTrainer pid=46417) [6]	train-rmse:1.11108	validation-rmse:1.11337
(XGBoostTrainer pid=46417) [7]	train-rmse:1.10987	validation-rmse:1.11221
(XGBoostTrainer pid=46417) [8]	train-rmse:1.10893	validation-rmse:1.11136
(XGBoostTrainer pid=46417) [9]	train-rmse:1.10832	validation-rmse:1.11088


(RayTrainWorker pid=46548) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-56-55/XGBoostTrainer_35e72_00003_3_eta=0.5000,max_depth=5_2025-11-20_22-57-49/checkpoint_000000)
(RayTrainWorker pid=46548) Registered dataset logger for dataset dataset_32_0 [repeated 3x across cluster]
(SplitCoordinator pid=46677) Starting execution of Dataset validation_29_0. Full logs are in /tmp/ray/session_2025-11-20_22-33-41_618501_40643/logs/ray-data
(SplitCoordinator pid=46677) Execution plan of Dataset validation_29_0: InputDataBuffer[Input] -> OutputSplitter[split(1, equal=True)]
(SplitCoordinator pid=46677) ✔️  Dataset validation_29_0 execution finished in 0.15 seconds


In [3]:
# load the preprocessed dataset as dense vectors in the parquet format
train_data_path=os.path.expanduser("~/public/pa3/ml_features_train.parquet")
train_data = ray.data.read_parquet(train_data_path)

Parquet dataset sampling 0:   0%|          | 0.00/1.00 [00:00<?, ? file/s]

2025-11-20 22:33:48,082	INFO parquet_datasource.py:699 -- Estimated parquet encoding ratio is 4.203.
2025-11-20 22:33:48,084	INFO parquet_datasource.py:759 -- Estimated parquet reader batch size at 677868 rows


### Training
Train a regression model (MSE objective) to predict `overall` using `XGBoostTrainer`.
- Objective: regression with squared error (`reg:squarederror`)
- Model params: `max_depth=3`, `eta=0.3`, others default.
- Use Ray Train with a suitable `ScalingConfig`; and you can set 2 CPUs per worker.
- After training, run inference on the test set.

Note: Ray will by default try to store results in `~/ray_results`. This can throw permission errors in DataHub, so you can change the location to `~/private/ray_results` using [RunConfig](https://docs.ray.io/en/latest/train/api/doc/ray.train.RunConfig.html).

In [4]:
# create the XGBoost trainer for regression on the "overall" label using the built-in trainer
import xgboost as xgb

trainer = XGBoostTrainer(
    label_column="overall",
    params={
        "objective": "reg:squarederror",
        "max_depth": 3,
        "eta": 0.3,
    },
    scaling_config=ScalingConfig(
        num_workers=1,
        use_gpu=False,
        resources_per_worker={"CPU": 2},
    ),
    run_config=RunConfig(storage_path="~/private/ray_results"),
    datasets={"train": train_data},
)

In [5]:
result = trainer.fit()

2025-11-20 22:33:48,689	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


== Status ==
Current time: 2025-11-20 22:33:50 (running for 00:00:00.12)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-11-20_22-33-41_618501_40643/artifacts/2025-11-20_22-33-48/XGBoostTrainer_2025-11-20_22-33-48/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-11-20 22:33:55 (running for 00:00:05.18)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-11-20_22-33-41_618501_40643/artifacts/2025-11-20_22-33-48/XGBoostTrainer_2025-11-20_22-33-48/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




(pid=43041) Running 0: 0.00 row [00:00, ? row/s]

(pid=43041) - ReadParquet->SplitBlocks(100) 1: 0.00 row [00:00, ? row/s]

(pid=43041) - split(1, equal=True) 2: 0.00 row [00:00, ? row/s]

== Status ==
Current time: 2025-11-20 22:34:00 (running for 00:00:10.21)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-11-20_22-33-41_618501_40643/artifacts/2025-11-20_22-33-48/XGBoostTrainer_2025-11-20_22-33-48/driver_artifacts
Number of trials: 1/1 (1 RUNNING)


== Status ==
Current time: 2025-11-20 22:34:05 (running for 00:00:15.23)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-11-20_22-33-41_618501_40643/artifacts/2025-11-20_22-33-48/XGBoostTrainer_2025-11-20_22-33-48/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




2025-11-20 22:34:08,612	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-33-48' in 0.0087s.
2025-11-20 22:34:08,615	INFO tune.py:1041 -- Total run time: 19.93 seconds (18.34 seconds for the tuning loop).


== Status ==
Current time: 2025-11-20 22:34:08 (running for 00:00:18.35)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/8 CPUs, 0/0 GPUs
Result logdir: /tmp/ray/session_2025-11-20_22-33-41_618501_40643/artifacts/2025-11-20_22-33-48/XGBoostTrainer_2025-11-20_22-33-48/driver_artifacts
Number of trials: 1/1 (1 TERMINATED)




In [6]:
print(result)

Result(
  metrics={'train-rmse': 1.120708220781679},
  path='/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-33-48/XGBoostTrainer_fb7d8_00000_0_2025-11-20_22-33-50',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-33-48/XGBoostTrainer_fb7d8_00000_0_2025-11-20_22-33-50/checkpoint_000000)
)


## Analyzing test data performance

Next, use the trained model to generate predictions on test data. Calculate the root mean square error (RMSE) of
the test predictions and report it in the output. 

For this task, we will make use of [`map_batches`](https://docs.ray.io/en/latest/data/api/doc/ray.data.Dataset.map_batches.html) to make a stateful transformation of the test data. 

In [7]:
test_data_path=os.path.expanduser("~/public/pa3/ml_features_test.parquet")
test_dataset= ray.data.read_parquet(test_data_path)

Parquet dataset sampling 0:   0%|          | 0.00/1.00 [00:00<?, ? file/s]

2025-11-20 22:34:08,733	INFO parquet_datasource.py:699 -- Estimated parquet encoding ratio is 3.786.
2025-11-20 22:34:08,734	INFO parquet_datasource.py:759 -- Estimated parquet reader batch size at 677868 rows


In [8]:
model = trainer.get_model(result.checkpoint)

In [9]:
import pandas as pd
from ray.train import Checkpoint
import xgboost
import math

class Predictor:

    def __init__(self, checkpoint: Checkpoint):
        # Load model from the provided checkpoint
        self.model = XGBoostTrainer.get_model(checkpoint)
        self.label_col = "overall"

    def __call__(self, batch: pd.DataFrame) -> pd.DataFrame:
        """
        1. Get the predictions on a batch of data for an xgboost model.
        2. Return the squared errors for each entry using the label column
        """
        batch_features, batch_labels =  batch.loc[:, batch.columns != self.label_col], batch[[self.label_col]]
        dmatrix = xgboost.DMatrix(batch_features)
        batch["predictions"] = self.model.predict(dmatrix)
        errors = (batch["predictions"] - batch_labels[self.label_col])**2
        return {"se": errors}

def predict_xgboost(test_dataset, result):
    """
    Obtains the predictions for a test dataset given a `ray.train.Result` object and returns the squared errors for each entry
    Hint: ds.map_batches()
    """
    # create predictor from the result checkpoint
    predictor = Predictor(result.checkpoint)

    # apply predictor to test dataset (batch-wise) and collect squared errors
    se_ds = test_dataset.map_batches(predictor, batch_format="pandas")
    se_df = se_ds.to_pandas()

    # return squared errors as a 1-D numpy array
    squared_errors = se_df["se"].to_numpy()
    return squared_errors

In [10]:
# get the root mean squared error for the test dataset using the result.
# Save the test rmse in `test_rmse` 

# Use the predictor implemented earlier to get squared errors, then compute RMSE
squared_errors = predict_xgboost(test_dataset, result)
test_rmse = float(np.sqrt(np.mean(squared_errors)))

# write to file
res_2_1 = {"test_rmse": test_rmse, 
          "train_rmse": result.metrics["train-rmse"]}

2025-11-20 22:34:09,895	INFO logging.py:293 -- Registered dataset logger for dataset dataset_5_0
2025-11-20 22:34:09,926	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_5_0. Full logs are in /tmp/ray/session_2025-11-20_22-33-41_618501_40643/logs/ray-data
2025-11-20 22:34:09,927	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_5_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> TaskPoolMapOperator[MapBatches(Predictor)]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->SplitBlocks(75) 1: 0.00 row [00:00, ? row/s]

- MapBatches(Predictor) 2: 0.00 row [00:00, ? row/s]

2025-11-20 22:34:13,539	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_5_0 execution finished in 3.61 seconds


In [11]:
expected_path = Path("~/public/pa3/expected_2_1.json").expanduser()
with open(expected_path) as expected_file:
    expected = json.load(expected_file)

assert math.isclose(float(expected["train_rmse"]), float(res_2_1["train_rmse"]), abs_tol=0.01), \
    f"train_rmse mismatch: expected {expected['train_rmse']} vs {res_2_1['train_rmse']} (±0.01)"

assert math.isclose(float(expected["test_rmse"]), float(res_2_1["test_rmse"]), abs_tol=0.01), \
    f"test_rmse mismatch: expected {expected['test_rmse']} vs {res_2_1['test_rmse']} (±0.01)"

print("✅ Task 2.1 output matches expected within ±0.01 for RMSE.")


✅ Task 2.1 output matches expected within ±0.01 for RMSE.


# Part 2: Tuning with Ray Tune

Based on `XGBoostTrainer` you just implemented, now We'll tune the following hyperparameters:

1. `max_depth`
2. `eta`

You can read more about each hyperparameter in the [official docs](https://xgboost.readthedocs.io/en/stable/parameter.html). Since the overall search space is large, and our compute budget is limited, we'll focus on running 4 *trials* (or 4 instances of 2-tuples of hyperparameters) with a grid search.  Here are the values:

1. `max_depth`: $[3, 5]$ 
2. `eta`: $[0.3, 0.5]$

Steps to implement, repeated from the problem statement:
1. Create a new training and validation data from the original training data - with a random split of 75/25.
2. Train Xgboost models with 4 hyperparameter trials over the given grid using Ray Tune. [Offical Example](https://docs.ray.io/en/latest/tune/examples/tune-xgboost.html)
3. Select the best model with the lowest validation RMSE. 
4. Report the test RMSE for the best model and the lowest validation RMSE.

Make sure to use the same `ScalingConfig` as before. Restrict the number of concurrent trials to 1 for memory efficiency. Store the final `tune.ResultGrid` object in `result_grid` and the best result in the variable `best_result`.

In [13]:
from ray import tune

# store your answers in these
best_result = None
result_grid = None

# Split the dataset into training and validation sets (75/25 split)
train_ds, valid_ds = train_data.train_test_split(test_size=0.25, seed=seed)

# Define the XGBoostTrainer with static configurations
trainer = XGBoostTrainer(
    label_column="overall",
    params={
        "objective": "reg:squarederror",
    },
    scaling_config=ScalingConfig(
        num_workers=1,
        use_gpu=False,
        resources_per_worker={"CPU": 2},
    ),
    run_config=RunConfig(storage_path="~/private/ray_results"),
    datasets={"train": train_ds, "validation": valid_ds},
)

# Define the Tuner for hyperparameter grid search
tuner = tune.Tuner(
    trainer,
    param_space={
        "params": {
            "max_depth": tune.grid_search([3, 5]),
            "eta": tune.grid_search([0.3, 0.5]),
        }
    },
    tune_config=tune.TuneConfig(
        max_concurrent_trials=1,
        metric="validation-rmse",
        mode="min",
    ),
)

# Execute the tuning process
result_grid = tuner.fit()

# Retrieve the best result based on validation RMSE
best_result = result_grid.get_best_result(metric="validation-rmse", mode="min")

(pid=45016) Running 0: 0.00 row [00:00, ? row/s]

(pid=45016) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

(pid=45017) Running 0: 0.00 row [00:00, ? row/s]

(pid=45017) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

(pid=45573) Running 0: 0.00 row [00:00, ? row/s]

...

(pid=46678) Running 0: 0.00 row [00:00, ? row/s]

(pid=46678) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

(pid=46677) Running 0: 0.00 row [00:00, ? row/s]

(pid=46677) - split(1, equal=True) 1: 0.00 row [00:00, ? row/s]

2025-11-20 22:58:08,299	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-56-55' in 0.0363s.
2025-11-20 22:58:08,303	INFO tune.py:1041 -- Total run time: 73.12 seconds (73.07 seconds for the tuning loop).


In [14]:
print(best_result)

Result(
  metrics={'train-rmse': 1.108322672118614, 'validation-rmse': 1.1108811485952415},
  path='/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-56-55/XGBoostTrainer_35e72_00003_3_eta=0.5000,max_depth=5_2025-11-20_22-57-49',
  filesystem='local',
  checkpoint=Checkpoint(filesystem=local, path=/home/wew068/private/ray_results/XGBoostTrainer_2025-11-20_22-56-55/XGBoostTrainer_35e72_00003_3_eta=0.5000,max_depth=5_2025-11-20_22-57-49/checkpoint_000000)
)


Now, 
1. Get the root mean squared error for the test dataset using the best result from the hyperparameter tuning experiments.
2. Report the validation rmse values for the best model as well as the given specific configurations

In [17]:
def get_task_2_2_results(result_grid: tune.ResultGrid, best_result: ray.train.Result):
    res = {
        "test_rmse": None, # test rmse for the best model
        "valid_rmse": None, # validation rmse for the best model
        "valid_depth_5_eta_0.3": None, # validation rmse for max_depth=5, eta=0.3
        "valid_depth_3_eta_0.5": None, # validation rmse for max_depth=3, eta=0.5
    }

    # compute test rmse using the supplied best_result
    # best_result is a ray.train.Result - we can use its checkpoint to predict on the test set
    test_se = predict_xgboost(test_dataset, best_result)
    res['test_rmse'] = float(np.sqrt(np.mean(test_se)))

    # validation rmse for best model
    res['valid_rmse'] = float(best_result.metrics['validation-rmse'])

    # helper: iterate over results in the result_grid and extract validation rmse for specific configs
    # ResultGrid is iterable, yielding ray.train.Result-like objects
    for r in result_grid:
        # r.config may contain the nested 'params' mapping used by XGBoostTrainer
        cfg = getattr(r, 'config', {}) or {}
        params = cfg.get('params', {})
        md = params.get('max_depth')
        eta = params.get('eta')

        valid = None
        # metrics may be stored on r.metrics
        if hasattr(r, 'metrics'):
            valid = r.metrics.get('validation-rmse')
        elif hasattr(r, 'result') and isinstance(r.result, dict):
            valid = r.result.get('validation-rmse')

        if md == 5 and eta == 0.3:
            res['valid_depth_5_eta_0.3'] = float(valid)
        if md == 3 and eta == 0.5:
            res['valid_depth_3_eta_0.5'] = float(valid)

    return res

In [18]:
res_2_2 = get_task_2_2_results(result_grid, best_result)

2025-11-20 23:02:38,392	INFO logging.py:293 -- Registered dataset logger for dataset dataset_33_0
2025-11-20 23:02:38,497	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_33_0. Full logs are in /tmp/ray/session_2025-11-20_22-33-41_618501_40643/logs/ray-data
2025-11-20 23:02:38,498	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_33_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> TaskPoolMapOperator[MapBatches(Predictor)]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->SplitBlocks(75) 1: 0.00 row [00:00, ? row/s]

- MapBatches(Predictor) 2: 0.00 row [00:00, ? row/s]

2025-11-20 23:02:40,273	INFO streaming_executor.py:279 -- ✔️  Dataset dataset_33_0 execution finished in 1.77 seconds


In [19]:
expected_path = Path("~/public/pa3/expected_2_2.json").expanduser()
with open(expected_path) as expected_file:
    expected = json.load(expected_file)

for key in res_2_2.keys():
    assert math.isclose(float(expected[key]), float(res_2_2[key]), abs_tol=0.01), \
        f"{key} mismatch: expected {expected[key]} vs {res_2_2[key]} (±0.01)"

print("✅ Task 2.2 output matches expected within ±0.01 for RMSE.")


✅ Task 2.2 output matches expected within ±0.01 for RMSE.


In [20]:
# shutdown!
ray.shutdown()